# Modelos de valuación de activos (CAPM y Multifactor) — Notebook de clase — SOLUCIÓN

**Objetivo:** estimar y comparar CAPM vs un modelo multifactor (MKT, SMB, HML, MOM), interpretar betas/alphas y decidir si hay evidencia de “alpha real” o solo riesgo disfrazado.

**Dataset:** `dataset_asset_pricing_mcf.csv` (incluye RF, mercado, factores y 5 activos).

In [ ]:
# === Setup ===
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

pd.set_option("display.float_format", lambda x: f"{x:,.6f}")

DATA_PATH = "dataset_asset_pricing_mcf.csv"  # si no está en el mismo folder, ajusta ruta

In [ ]:
# === Carga de datos ===
df = pd.read_csv(DATA_PATH, parse_dates=["date"]).sort_values("date").reset_index(drop=True)

# Vista rápida
display(df.head())
print("Filas/Columnas:", df.shape)
print("Rango fechas:", df["date"].min().date(), "→", df["date"].max().date())

## 1) Warm-up: retornos exceso y exploración rápida

**Idea:** En CAPM y modelos de factores normalmente trabajamos con *retornos en exceso*:
- `ASSET_EXCESS = ASSET - RF`
- `MKT_EXCESS = MKT - RF`

**Tarea:** crea las columnas de retornos en exceso para:
- Mercado (`MKT_EXCESS`)
- Un activo que tú elijas (por ejemplo `ASSET_A`)

Luego calcula:
- media y volatilidad anualizada (aprox. 252 días hábiles)

In [ ]:
df["MKT_EXCESS"] = df["MKT"] - df["RF"]
ASSET_COL = "ASSET_A"
df["ASSET_EXCESS"] = df[ASSET_COL] - df["RF"]

PER_YEAR = 252
def annualize_mean(r, periods_per_year=PER_YEAR):
    return r.mean() * periods_per_year

def annualize_vol(r, periods_per_year=PER_YEAR):
    return r.std() * np.sqrt(periods_per_year)

print("E[MKT_EXCESS] anual:", annualize_mean(df["MKT_EXCESS"]))
print("Vol[MKT_EXCESS] anual:", annualize_vol(df["MKT_EXCESS"]))
print("E[ASSET_EXCESS] anual:", annualize_mean(df["ASSET_EXCESS"]))
print("Vol[ASSET_EXCESS] anual:", annualize_vol(df["ASSET_EXCESS"]))

## 2) Mini-Markowitz (opcional, rápido)

No vamos a optimizar con restricciones formales aquí, pero sí ver la intuición:

**Tarea:** construye 5,000 portafolios aleatorios con los 5 activos (`ASSET_A` a `ASSET_E`), calcula retorno y volatilidad anualizada y grafica la nube.

> Si no te da tiempo: salta a CAPM. No te juzgo (mucho).

In [ ]:
asset_cols = ["ASSET_A","ASSET_B","ASSET_C","ASSET_D","ASSET_E"]
rets = df[asset_cols]

W = np.random.dirichlet(np.ones(len(asset_cols)), size=5000)

mu = rets.mean().values
Sigma = rets.cov().values

port_mu = W @ mu
port_var = np.einsum('ij,jk,ik->i', W, Sigma, W)
port_vol = np.sqrt(port_var)

mu_a = port_mu * 252
vol_a = port_vol * np.sqrt(252)

plt.figure()
plt.scatter(vol_a, mu_a, s=6)
plt.xlabel("Volatilidad anual")
plt.ylabel("Retorno anual esperado (aprox.)")
plt.title("Nube de portafolios aleatorios (5 activos)")
plt.show()

## 3) CAPM: estimación e interpretación

Modelo:
\[
R_i - R_f = \alpha + \beta (R_m - R_f) + \epsilon
\]

**Tareas:**
1) Ajusta el modelo CAPM (OLS) para tu activo elegido.  
2) Reporta: alpha, beta, p-values, R².  
3) Interpreta en 4–6 líneas: ¿qué significa el beta? ¿hay evidencia de alpha (p-value)?

In [ ]:
X = sm.add_constant(df["MKT_EXCESS"])
y = df["ASSET_EXCESS"]

capm = sm.OLS(y, X).fit()
print(capm.summary())

alpha = capm.params["const"]
beta = capm.params["MKT_EXCESS"]
p_alpha = capm.pvalues["const"]
p_beta = capm.pvalues["MKT_EXCESS"]
r2 = capm.rsquared

print("\nResumen CAPM")
print("alpha (diario):", alpha)
print("beta:", beta)
print("p(alpha):", p_alpha)
print("p(beta):", p_beta)
print("R²:", r2)

## 4) Multifactor (MKT, SMB, HML, MOM)

Modelo:
\[
R_i - R_f = \alpha + b_m (R_m - R_f) + b_{smb} SMB + b_{hml} HML + b_{mom} MOM + \epsilon
\]

**Tareas:**
1) Ajusta el multifactor.  
2) Compara contra CAPM: R², alpha y significancia.  
3) Interpreta al menos 2 factores (ej. SMB positivo = sesgo a acciones pequeñas).

In [ ]:
factors = ["MKT_EXCESS", "SMB", "HML", "MOM"]
Xk = sm.add_constant(df[factors])
mf = sm.OLS(df["ASSET_EXCESS"], Xk).fit()
print(mf.summary())

def summarize_model(m):
    return pd.DataFrame({"coef": m.params, "pval": m.pvalues})

capm_tbl = summarize_model(capm)
mf_tbl = summarize_model(mf)

display(pd.concat({"CAPM": capm_tbl, "MF": mf_tbl}, axis=1))
print("CAPM R²:", capm.rsquared)
print("MF   R²:", mf.rsquared)

## 5) Reporte final (mini-caso)

Entrega en el notebook (o en texto aquí):
- Tabla CAPM vs MF (coeficientes y p-values)
- CAPM R² vs MF R²
- Conclusión ejecutiva:
  - ¿Hay alpha? (basado en p-value + magnitud)
  - ¿Qué factores dominan el retorno en exceso?
  - 1 riesgo/limitación (estabilidad, sobreajuste, régimen, etc.)

In [ ]:
# Conclusión ejemplo (ajusta según tu activo):
# - Beta > 1 sugiere mayor sensibilidad al mercado.
# - Si alpha no es significativo (p alto), no hay evidencia de retorno extra consistente.
# - Multifactor suele subir R² porque captura exposiciones estilo.

## 6) Bonus (+10): estabilidad (rolling beta)

**Objetivo:** estimar beta en ventanas móviles (ej. 252 días) para ver si es estable o cambia por régimen.

Tarea:
- Calcula beta rolling (regresión por ventana) y grafícala.

In [ ]:
window = 252
roll_cov = df["ASSET_EXCESS"].rolling(window).cov(df["MKT_EXCESS"])
roll_var = df["MKT_EXCESS"].rolling(window).var()
roll_beta = roll_cov / roll_var

plt.figure()
plt.plot(df["date"], roll_beta)
plt.title("Rolling beta (ventana 252)")
plt.xlabel("Fecha")
plt.ylabel("Beta")
plt.show()

print("Beta rolling: últimos valores")
display(roll_beta.dropna().tail())